In [ ]:
#!pip install langchain langchain-ollama langchain-text-splitters langchain-community chromadb pypdf faiss-gpu faiss-cpu

In [4]:
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# Carregar o PDF, Extrair as informações e dividir em chunks
docs = PyPDFLoader("data/lei-8080.pdf").load()
chunks = RecursiveCharacterTextSplitter(chunk_size=800, 
                                        chunk_overlap=120).split_documents(docs)

In [25]:
# Embeddings locais + FAISS
emb = OllamaEmbeddings(model="nomic-embed-text")

vstore = FAISS.from_documents(chunks, 
                              embedding=emb)

retriever = vstore.as_retriever(search_type="similarity", 
                                search_kwargs={"k": 10})

In [19]:
# Modelo local via Ollama
llm = OllamaLLM(model="llama3:8b")

In [ ]:
query = "Quais os princípios do SUS segundo a Lei 8.080?"


docs = retriever.get_relevant_documents(query)
ctx = "\n\n".join(d.page_content[:900] for d in docs)
resp = llm.invoke(f"Responda com base no texto:\n\n{ctx}\n\nPergunta: {query}")
print(resp)

Segundo a Lei nº 8.080, de 19 de setembro de 1990, os princípios do Sistema Único de Saúde (SUS) são:

I - universalidade de acesso aos serviços de saúde em todos os níveis de assistência;

XIII - organização dos serviços públicos de modo a evitar duplicidade de meios para fins idênticos;

XIV - organização de atendimento público específico e especializado para mulheres e vítimas de violência doméstica em geral, que garanta, entre outros, atendimento, acompanhamento psicológico e cirurgias plásticas reparadoras.

Esses princípios devem ser observados na gestão e desenvolvimento das ações e serviços públicos de saúde no SUS.
